In [1]:
import requests

In [28]:
url = "http://localhost:8000/ingest"
file_path = "data/testPDFs/BERT.pdf"

# We open the file in binary mode
with open(file_path, "rb") as f:
    files = {"file": f}
    # We send the model choice as form data
    data = {"model_name": "bert"}

    print(f"📤 Uploading {file_path}...")
    response = requests.post(url, files=files, data=data)

if response.status_code == 200:
    print("✅ Ingestion Success!")
    print(response.json())
else:
    print(f"❌ Error {response.status_code}: {response.text}")

📤 Uploading data/testPDFs/BERT.pdf...
✅ Ingestion Success!
{'filename': 'BERT.pdf', 'message': 'Ingestion successful', 'chunks_added': 31, 'parent_id': 'arXiv:1810.04805v2'}


In [29]:
url = "http://localhost:8000/query"
payload = {
    "query_text": "What are the architecture details of the transformer?",
    "n_results": 2,
    "model_name": "bert"
}

print(f"🔍 Searching for: '{payload['query_text']}'")
response = requests.post(url, json=payload)

if response.status_code == 200:
    data = response.json()
    results = data.get("results", [])

    print(f"✅ Found {len(results)} matches:\n")
    for res in results:
        print(f"📄 Doc ID: {res['doc_id']}")
        print(f"📊 Score:  {res['score']:.4f}")
        print(f"📝 Content: {res['content'][:100]}...") # Preview first 100 chars
        print("-" * 30)
else:
    print(f"❌ Error {response.status_code}: {response.text}")

🔍 Searching for: 'What are the architecture details of the transformer?'
✅ Found 2 matches:

📄 Doc ID: arXiv:1810.04805v2#4_Experiments
📊 Score:  0.4823
📝 Content: In this section, we present BERT fine-tuning results on 11 NLP tasks....
------------------------------
📄 Doc ID: arXiv:1810.04805v2#2.2_Unsupervised_Fine-tuning_Approaches
📊 Score:  0.4918
📝 Content: As with the feature-based approaches, the first works in this direction only pre-trained word embedd...
------------------------------


In [30]:
# Deleting specific documents
url = "http://localhost:8000/delete"
payload = {
    "model_name": "bert",
    "doc_ids": ["arXiv_1706.03762v7#Introduction", "arXiv_1706.03762v7#Conclusion"]
}
requests.post(url, json=payload)

<Response [200]>

In [32]:
# Resetting the entire collection for a model
url = "http://localhost:8000/reset"
payload = {"model_name": "bert"}
requests.post(url, json=payload)

<Response [200]>

In [31]:
# List items in the collection
url = "http://localhost:8000/list-ids"

params = {
    "model_name": "bert",
    "limit": 5
}

response = requests.get(url, params=params)
print(response.json())

{'model_name': 'bert', 'ids': ['arXiv:1810.04805v2#Preamble', 'arXiv:1810.04805v2#BERT:_Pre-training_of_Deep_Bidirectional_Transform', 'arXiv:1810.04805v2#Abstract', 'arXiv:1810.04805v2#1_Introduction', 'arXiv:1810.04805v2#2_Related_Work'], 'total_in_batch': 5}


In [33]:
# Debug embeddings for a specific text
url = "http://localhost:8000/debug/embed"
payload = {
    "text": "The transformer architecture allows for parallelization.",
    "model_name": "bert"
}
response = requests.post(url, json=payload)
print(response.json()["dimension"])

768


In [34]:
# Parse a PDF and get JSON
url = "http://localhost:8000/debug/parse-pdf"
files = {'file': open('./data/testPDFs/BERT.pdf', 'rb')}

response = requests.post(url, files=files)
data = response.json()

print("Title detected:", data['metadata_extracted']['title'])
print("Sections found:", list(data['sections'].keys()))

Title detected: BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding
Sections found: ['Preamble', 'BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding', 'Abstract', '1 Introduction', '2 Related Work', '2.1 Unsupervised Feature-based Approaches', '2.2 Unsupervised Fine-tuning Approaches', '2.3 Transfer Learning from Supervised Data', '3 BERT', '3.1 Pre-training BERT', '3.2 Fine-tuning BERT', '4 Experiments', '4.1 GLUE', '4.2 SQuAD v1.1', '4.3 SQuAD v2.0', '4.4 SWAG', '5 Ablation Studies', '5.1 Effect of Pre-training Tasks', '5.2 Effect of Model Size', '5.3 Feature-based Approach with BERT', '6 Conclusion', 'References', "Appendix for 'BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding'", 'A.1 Illustration of the Pre-training Tasks', 'A.2 Pre-training Procedure', 'A.3 Fine-tuning Procedure', 'A.4 Comparison of BERT, ELMo ,and OpenAI GPT', 'A.5 Illustrations of Fine-tuning on Different Tasks', 'B.1 Deta